# From NIR to SPECK

In this notebook we will show how a NIR model can be deployed onto SynSense's Speck processor. 

We start by importing the model into sinabs and from there deploy it onto Speck. We will test the deployed model by streaming events onto the chip and reading out the predictions.

In [1]:
# Import statements
import sinabs
import nir

ModuleNotFoundError: No module named 'sinabs'

## Load NIR model from disk

In [ ]:
model_path = "nmnist_model.nir"
nir_model = ...

## Load NIR model into sinabs

Loading a NIR model into sinabs is fairly easy: It's literally one line of code.

In [ ]:
model = sinabs.from_nir(nir_model, batch_size=1)

### Test the model in software

Let's quickly run the test set through the sinabs model to make sure everything works as it should.

In [ ]:
dataset = ...
dataloader = torch.utils.data.Dataloader(dataset, batch_size=1)

correct = 0
for i, (sample, target) in tqdm(enumerate(dataloader)):
    output = model(sample)
    prediction = torch.argmax(output.sum(0))
    correct += (prediction == target)

accuracy = correct / len(dataset)
print(f"Test accuracy with sinabs model: {accuracy:.2%}")

## Deploy the model onto Speck

Deploying a model onto Speck with Sinabs is also straightforward:

In [ ]:
dynapcnn_model = sinabs.backend.dynapcnn.DynapcnnNetwork(model)
dynapcnn_model.to("speck2fmodule")

## Send test data to the chip and read out its prediction
Now the model that we imported from NIR has been deployed onto Speck and is ready to run. Let's see if it works.